In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pickle

# Charger les données depuis MongoDB pour entraîner le modèle
def load_data_from_mongo():
    planets_cursor = planets_collection.find()
    planets = list(planets_cursor)
    for planet in planets:
        planet['_id'] = str(planet['_id'])
    return pd.DataFrame(planets)

def train_model():
    data = load_data_from_mongo()

    # Sélection des caractéristiques pertinentes pour l'entraînement
    X = data[['Num_Moons', 'Minerals', 'Gravity', 'Sunlight_Hours', 'Temperature', 'Rotation_Time', 'Water_Presence']]
    y = data['Colonisable']

    # Séparation des données en ensemble d'entraînement et de test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Entraîner un modèle de régression logistique
    model = LogisticRegression()
    model.fit(X_train, y_train)

    # Sauvegarder le modèle pour une utilisation ultérieure
    with open('colonisation_model.pkl', 'wb') as f:
        pickle.dump(model, f)

    print("Modèle entraîné et sauvegardé.")


In [ ]:
# Charger le modèle lors du démarrage de l'application
def load_model():
    with open('colonisation_model.pkl', 'rb') as f:
        model = pickle.load(f)
    return model

model = load_model()


In [ ]:
@app.route('/api/planet/predict', methods=['POST'])
def predict_colonisability():
    try:
        data = request.get_json()

        # Vérification que toutes les caractéristiques nécessaires sont présentes
        required_fields = ['Num_Moons', 'Minerals', 'Gravity', 'Sunlight_Hours', 'Temperature', 'Rotation_Time', 'Water_Presence']
        for field in required_fields:
            if field not in data:
                return jsonify({"error": f"Le champ '{field}' est manquant."}), 400

        # Extraire les caractéristiques pour la prédiction
        features = [[
            data['Num_Moons'],
            data['Minerals'],
            data['Gravity'],
            data['Sunlight_Hours'],
            data['Temperature'],
            data['Rotation_Time'],
            data['Water_Presence']
        ]]

        # Faire la prédiction avec le modèle chargé
        prediction = model.predict(features)[0]
        probability = model.predict_proba(features)[0][1]  # Probabilité d'être colonisable

        # Retourner le résultat
        return jsonify({
            "colonisable": bool(prediction),
            "probability": probability
        }), 200

    except Exception as e:
        return jsonify({"error": str(e)}), 500


In [ ]:
@app.route('/api/planet/predict', methods=['POST'])
def predict_colonisability():
    try:
        data = request.get_json()

        # Vérification que toutes les caractéristiques nécessaires sont présentes
        required_fields = ['Num_Moons', 'Minerals', 'Gravity', 'Sunlight_Hours', 'Temperature', 'Rotation_Time', 'Water_Presence']
        for field in required_fields:
            if field not in data:
                return jsonify({"error": f"Le champ '{field}' est manquant."}), 400

        # Extraire les caractéristiques pour la prédiction
        features = [[
            data['Num_Moons'],
            data['Minerals'],
            data['Gravity'],
            data['Sunlight_Hours'],
            data['Temperature'],
            data['Rotation_Time'],
            data['Water_Presence']
        ]]

        # Faire la prédiction avec le modèle chargé
        prediction = model.predict(features)[0]
        probability = model.predict_proba(features)[0][1]  # Probabilité d'être colonisable

        # Retourner le résultat
        return jsonify({
            "colonisable": bool(prediction),
            "probability": probability
        }), 200

    except Exception as e:
        return jsonify({"error": str(e)}), 500
